In [1]:
import pandas as pd
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [3]:
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [4]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

In [5]:
def get5minutedata(symbol, interval):
    frame = pd.DataFrame(session_auth.query_kline(symbol=symbol, interval=interval)["result"])
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [6]:
df = get5minutedata("ETHUSDT", "5m")
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-24 04:20:00,1637.36,1640.36,1636.80,1640.36,42.15284
2023-01-24 04:25:00,1640.36,1640.68,1637.08,1638.62,28.44273
2023-01-24 04:30:00,1638.62,1638.62,1636.97,1636.98,18.59142
2023-01-24 04:35:00,1636.98,1636.99,1636.12,1636.84,39.59181
2023-01-24 04:40:00,1636.84,1637.16,1636.51,1636.79,43.01813
...,...,...,...,...,...
2023-01-27 15:15:00,1588.55,1591.21,1582.95,1584.36,457.51559
2023-01-27 15:20:00,1584.36,1585.19,1581.00,1583.74,301.90594
2023-01-27 15:25:00,1583.74,1585.09,1581.90,1585.09,219.29544


In [68]:
eth = pd.read_csv ('./eth_actualizado.csv')
eth['Time'] = eth['time'].str.extract('(\d{4}-\d{2}-\d{2}\w\d{2}:\d{2}:\d{2})', expand=True)
eth['Time'] = pd.to_datetime(eth["Time"])
eth.set_index("Time", inplace = True)
eth.drop(columns="time", inplace=True)
eth.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
eth = eth[7000:]

In [69]:
eth

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-12 08:20:00,1399.95,1400.70,1399.50,1400.70,599.75
2023-01-12 08:25:00,1400.70,1401.85,1399.95,1400.00,1721.92
2023-01-12 08:30:00,1400.00,1401.55,1399.70,1400.10,2315.36
2023-01-12 08:35:00,1400.10,1401.05,1399.90,1400.45,442.07
2023-01-12 08:40:00,1400.45,1401.00,1396.50,1396.50,2346.02
...,...,...,...,...,...
2023-01-24 21:25:00,1618.74,1618.99,1616.19,1616.95,1628.70
2023-01-24 21:30:00,1616.95,1618.38,1616.28,1616.28,874.82
2023-01-24 21:35:00,1616.28,1617.17,1613.25,1613.57,3684.93


In [92]:
class ETH_Short(Strategy):

    def init(self):
        close = self.data.Close
        self.rsi= self.I(ta.momentum.rsi, pd.Series(close), window=14)
        self.K = self.I(ta.momentum.stochrsi_k, pd.Series(close), window= 14)
        self.D = self.I(ta.momentum.stochrsi_d, pd.Series(close), window= 14)

    def next(self):
        price = self.data.Close
        rsi = self.rsi
        K = self.K
        D = self.D
       
        if  rsi > 68 and K + 0.04 < D and D > 0.75:
            sl = price * 1.015
            tp = price * 0.96
            self.sell(sl=sl, tp=tp)
    

In [93]:
bt = Backtest(df, ETH_Short, cash= 10000, commission= 0.0015)

In [94]:
output = bt.run()

In [95]:
output

Start                     2023-01-24 04:20:00
End                       2023-01-27 15:35:00
Duration                      3 days 11:15:00
Exposure Time [%]                        61.9
Equity Final [$]                  9993.998885
Equity Peak [$]                   10109.18716
Return [%]                          -0.060011
Buy & Hold Return [%]               -3.219415
Return (Ann.) [%]                   -3.711219
Volatility (Ann.) [%]               42.281693
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.852039
Avg. Drawdown [%]                   -1.075047
Max. Drawdown Duration        2 days 00:25:00
Avg. Drawdown Duration        0 days 08:03:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       3.855784
Worst Trade [%]                     -1.652479
Avg. Trade [%]                    

In [96]:
bt.plot()

Row(id='18936', ...)